## Display LSSTCam images
In this notebook, we show how to query the LSSTCam repository\
and view the resulting images.\

Craig Lage - 07-Sep-24

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.math as afwMath
import lsst.afw.display as afwDisplay

In [ ]:
butler = Butler('embargo', collections=['LSSTCam/raw/all', 'LSSTCam/calib/unbounded'])

In [ ]:
dayObs = 20240909
instrument = "LSSTCam"

exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time, record.physical_filter)


## Define a simple ISR
### Just overscan subtraction.

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=False
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR on one CCD and look at the result

In [ ]:
instrument = 'LSSTCam'
camera = butler.get('camera', instrument=instrument)
expId = 2024090900006
detector = 2
exp = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
isrResult = isrTask.run(exp) # This runs the ISR

In [ ]:
%matplotlib inline
x = plot(isrResult.exposure, stretch='ccs')
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_{expId}_{detector}.png")

In [ ]:
def isrCallback(im, ccd, imageSource):
    """Assemble the CCD image and do basic ISR"""
    dayObs = imageSource.kwargs['day_obs']
    seqNum = imageSource.kwargs['seq_num']
    exp = imageSource.butler.get('raw', detector=ccd.getId(), day_obs=dayObs, seq_num=seqNum)
    isrResult = isrTask.run(exp)
    nQuarter = ccd.getOrientation().getNQuarter()
    oim = afwMath.rotateImageBy90(isrResult.exposure.image, nQuarter)
    return oim

def rawCallback(im, ccd, imageSource):
    """Assemble the CCD image.  Just bias subtraction and gain correction"""
    oim = camGeomUtils.rawCallback(im, ccd, imageSource,
                                   subtractBias=False, correctGain=False)
    return oim

In [ ]:
%matplotlib inline
instrument = "LSSTCam"
camera = butler.get('camera', instrument=instrument)
fig = plt.figure(figsize=(12,12))
disp = afwDisplay.Display(1, "matplotlib")
disp.scale('linear', min=0, max=100000)
dayObs = 20240906
seqNum = 117
dataType='raw'
mos = camGeomUtils.showCamera(camera,
                              camGeomUtils.ButlerImage(butler, dataType, 
                                                       instrument=instrument, 
                                                       day_obs=dayObs, seq_num=seqNum,
                                                       verbose=False, callback=rawCallback,
                                                       background=np.nan),
                              binSize=4, display=disp, overlay=False,
                              title="%d %d" % (dayObs, seqNum))

plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSSTCam_Raw_{dayObs}_{seqNum}.png")

In [ ]:
%matplotlib inline
plt.title(f"LSSTCam {dayObs}_{seqNum}")
plt.plot(mos.array[2000, :], label="X slice")
plt.plot(mos.array[:, 2000], label="Y slice")
plt.ylim(-5, 5)
#plt.ylim(-50, 50)
plt.ylabel("Flux (ADU)")
plt.xlabel("Pixels (binned by 16)")
plt.legend()
plt.savefig(f"/home/c/cslage/u/LSSTCam/images/LSST_Slices_{dayObs}_{seqNum}.png")